In [19]:
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from gensim.models import LdaModel, LsiModel
from gensim.corpora import Dictionary
from gensim import corpora
import pyLDAvis.gensim
import numpy as np

from nltk import word_tokenize
from nltk.corpus import stopwords
import string

import matplotlib.pyplot  as plt

In [5]:
file = pd.read_csv('lematizador.txt', sep = '\t', header = None)
file.columns = ['surfice', 'lemma']
file.set_index('surfice', inplace = True)
diccionario = file.to_dict()
dicc = diccionario['lemma']

In [6]:
stopwords = stopwords.words('spanish')+list(string.punctuation)

In [7]:
def lematizar_dic(lista, dicc):
    valores = []
    for x in lista:
        if x in dicc.keys() and x != 'estado':
            valores.append(dicc[x])
        else:
            valores.append(x)
    return(' '.join(valores))   

In [8]:
df = pd.read_pickle('mejor_lematizado.pkl')

In [9]:
textos = [word_tokenize(x.lower().replace('``','').replace("\'\'",'')) for x in df.texto]

In [10]:
lista2 = []
for x in textos:
    y = lematizar_dic(x, dicc)
    lista2.append(y)

In [11]:
lematizado_plenaria_df = pd.Series(lista2)

In [12]:
nueva_df = pd.concat([df,lematizado_plenaria_df.rename('texto_lematizado')], axis = 1).dropna(subset=['posicion'])#.groupby('posicion')

In [20]:
stopwords = stopwords.words('spanish')+list(string.punctuation)+list("\'\'", )

In [21]:
def procesar_texto(text, stopwords=[]):
    text = word_tokenize(text.lower())
    return [tok for tok in text if tok not in stopwords]

In [23]:
texts = [procesar_texto(x,stopwords) for x in nueva_df.texto] 

In [24]:
dictionary = corpora.Dictionary(texts)
print(len(dictionary))
dictionary.filter_extremes()
print(len(dictionary))

27991
6948


In [25]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [26]:
ldamodel = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary, iterations=2000, passes=10)

In [27]:
ldamodel.print_topics(10, 4)

[(0, '0.022*"derechos" + 0.014*"humanos" + 0.009*"derecho" + 0.009*"mujer"'),
 (1, '0.011*"mujer" + 0.009*"embarazo" + 0.008*"años" + 0.006*"puede"'),
 (2, '0.006*"parece" + 0.005*"creo" + 0.005*"muerte" + 0.005*"embarazo"'),
 (3, '0.010*"salud" + 0.007*"años" + 0.007*"embarazo" + 0.007*"mujer"'),
 (4, '0.020*"derecho" + 0.012*"derechos" + 0.009*"persona" + 0.006*"niño"'),
 (5, '0.009*"mujer" + 0.008*"\'\'" + 0.008*"humano" + 0.008*"embrion"'),
 (6, '0.021*"salud" + 0.008*"materna" + 0.008*"mortalidad" + 0.008*"ciento"'),
 (7, '0.006*"semanas" + 0.006*"quiero" + 0.005*"estan" + 0.005*"va"'),
 (8, '0.011*"derechos" + 0.011*"derecho" + 0.008*"salud" + 0.008*"ley"'),
 (9, '0.008*"madre" + 0.007*"años" + 0.007*"hoy" + 0.006*"embarazo"')]

In [28]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, mds="tsne")

TypeError: Could not operate array([481.,  49., 427., ..., 535., 468., 169.]) with block values unexpected indent (test_numexpr.py, line 1279)

In [14]:
#nueva_df.to_pickle('dos_lematizaciones.pkl')